# Collecting Our Education Status Data

## Import Libraries

In [1]:
import requests
import os
import io
import boto3

## Starting our AWS Session

In [2]:
session = boto3.Session(profile_name='personal-onemap')
s3 = session.resource('s3')

## Import API Authorisation Key

In [3]:
# For OneMap Account Key
from dotenv import load_dotenv
load_dotenv()
account_key = os.getenv('ONEMAP_KEY')
headers = {'Authorization': f'Bearer {account_key}'}

## Count number of areas in Singapore

In [4]:
api_url = "https://www.onemap.gov.sg/api/public/popapi/getPlanningareaNames?year=2019" # Latest info from 2019

response = requests.get(api_url, headers=headers, timeout=1)
data = response.json()

# Count number of area codes
area_count = sum(1 for item in data if "pln_area_n" in item)
print(f'Number of areas: {area_count}')

# Save our area codes into List
area_list = [item['pln_area_n'] for item in data if "pln_area_n" in item]
print(area_list)


Number of areas: 55
['BEDOK', 'BUKIT TIMAH', 'BUKIT BATOK', 'BUKIT MERAH', 'CENTRAL WATER CATCHMENT', 'DOWNTOWN CORE', 'CHANGI', 'CHANGI BAY', 'LIM CHU KANG', 'BOON LAY', 'WESTERN WATER CATCHMENT', 'WOODLANDS', 'MARINE PARADE', 'NEWTON', 'NORTH-EASTERN ISLANDS', 'ORCHARD', 'PASIR RIS', 'PIONEER', 'PUNGGOL', 'QUEENSTOWN', 'SEMBAWANG', 'SIMPANG', 'TAMPINES', 'TANGLIN', 'TUAS', 'WESTERN ISLANDS', 'SOUTHERN ISLANDS', 'BUKIT PANJANG', 'BISHAN', 'ANG MO KIO', 'GEYLANG', 'STRAITS VIEW', 'JURONG EAST', 'HOUGANG', 'JURONG WEST', 'CHOA CHU KANG', 'KALLANG', 'MANDAI', 'TENGAH', 'MARINA EAST', 'MARINA SOUTH', 'MUSEUM', 'NOVENA', 'OUTRAM', 'PAYA LEBAR', 'RIVER VALLEY', 'ROCHOR', 'SELETAR', 'SENGKANG', 'SERANGOON', 'CLEMENTI', 'TOA PAYOH', 'SINGAPORE RIVER', 'SUNGEI KADUT', 'YISHUN']


## Test API Output

In [5]:
response = requests.get(f"https://www.onemap.gov.sg/api/public/popapi/getEducationAttending?planningArea=Bedok&year=2020", headers=headers, timeout=1)
data = response.json()

print(response)
print(data)

<Response [200]>
[{'planning_area': 'Bedok', 'pre_primary': 3290, 'primary': 14572, 'secondary': 10517, 'post_secondary': 3309, 'polytechnic': 3923, 'prof_qualification_diploma': 580, 'university': 7785, 'year': 2020}]


In [6]:
years = ['2000', '2010', '2015', '2020']  # list of years available for data
genders = ['male', 'female']  

# Iterate over each year and gender
for year in years:
    for gender in genders:
        csv_file = io.StringIO()
        csv_file.write('planning_area,pre_primary,primary,secondary,post_secondary,polytechnic,prof_qualification_diploma,university,year\n')

        for area in area_list:
            response = requests.get(f"https://www.onemap.gov.sg/api/public/popapi/getEducationAttending?planningArea={area}&year={year}&gender={gender}", headers=headers, timeout=1.5)
            data = response.json()

            if response.status_code == 200:
                # Check for "No Data Available!" (When Area has not been created since 2019)
                if 'Result' in data and data['Result'] == "No Data Available!":
                    print(f"Skipped: {area} in {year} for {gender} has no results.")
                elif data:
                    for item in data:
                        csv_file.write(f"{item['planning_area']},{item['pre_primary']},{item['primary']},{item['secondary']},{item['post_secondary']},{item['polytechnic']},{item['prof_qualification_diploma']},{item['university']},{item['year']}\n")
                    print(f"Success: {area} in {year} for {gender} has been logged.")
            else:
                print(f"Failed to process {area} in {year} for {gender}.")

        print(f"All areas processed for {year} and {gender}.")

        # Move back to start of the file
        csv_file.seek(0)

        # S3 file path
        s3_file_path = f'Education_Status_Data/{year}/' +f'education_status_by_area_{year}_{gender}.csv'

        # Upload to S3
        s3.Object("onemap-s3", s3_file_path).put(Body=csv_file.getvalue(), ContentType='text/csv')
        csv_file.close()

        print(f"Successfully uploaded to S3: {s3_file_path}")


Success: BEDOK in 2000 for male has been logged.
Success: BUKIT TIMAH in 2000 for male has been logged.
Success: BUKIT BATOK in 2000 for male has been logged.
Success: BUKIT MERAH in 2000 for male has been logged.
Skipped: CENTRAL WATER CATCHMENT in 2000 for male has no results.
Success: DOWNTOWN CORE in 2000 for male has been logged.
Success: CHANGI in 2000 for male has been logged.
Skipped: CHANGI BAY in 2000 for male has no results.
Skipped: LIM CHU KANG in 2000 for male has no results.
Skipped: BOON LAY in 2000 for male has no results.
Skipped: WESTERN WATER CATCHMENT in 2000 for male has no results.
Success: WOODLANDS in 2000 for male has been logged.
Success: MARINE PARADE in 2000 for male has been logged.
Success: NEWTON in 2000 for male has been logged.
Skipped: NORTH-EASTERN ISLANDS in 2000 for male has no results.
Skipped: ORCHARD in 2000 for male has no results.
Success: PASIR RIS in 2000 for male has been logged.
Skipped: PIONEER in 2000 for male has no results.
Skipped: PU